# General Observing

This notebook is used for general observing with the Auxiliary Telescope.

Craig Lage - 25-May-21

In [1]:
import sys
import asyncio
import time
import os
import numpy as np

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [4]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.01 sec
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, algorithm)
Read 14 history items for RemoteEvent(ATDomeTrajectory, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, authList)
Read 3 history items for RemoteEvent(ATDomeTrajectory, 0, followingMode)
Read 100 history items for RemoteEvent(ATDomeTrajectory, 0, heartbeat)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, logLevel)
Read 38 history items for RemoteEvent(ATDomeTrajectory, 0, logMessage)
Read 1 history items for RemoteEvent(ATDomeTrajectory, 0, settingVersions)
Read 1 history it

[[None, None, None, None, None, None, None], [None, None, None, None]]

trajectory DDS read queue is filling: 33 of 100 elements
mountStatus DDS read queue is full (100 elements); data may be lost
m1AirPressure DDS read queue is filling: 19 of 100 elements
torqueDemand DDS read queue is filling: 33 of 100 elements
mountPositions DDS read queue is filling: 26 of 100 elements
loadCell DDS read queue is filling: 19 of 100 elements
currentTargetStatus DDS read queue is full (100 elements); data may be lost
nasymth_m3_mountMotorEncoders DDS read queue is filling: 33 of 100 elements
mount_Nasmyth_Encoders DDS read queue is filling: 33 of 100 elements
mount_AzEl_Encoders DDS read queue is filling: 33 of 100 elements
mount_AzEl_Encoders python read queue is filling: 32 of 100 elements
measuredTorque DDS read queue is filling: 33 of 100 elements
measuredMotorVelocity DDS read queue is filling: 33 of 100 elements
azEl_mountMotorEncoders DDS read queue is filling: 33 of 100 elements
target DDS read queue is filling: 12 of 100 elements
target python read queue is fill

In [ ]:
# enable components
#await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
#await latiss.enable({"atspectrograph": "current"})

In [ ]:
# If some components fail to enable, some set of commands like the ones below may be needed
# await salobj.set_summary_state(atcs.rem.atdome, salobj.State.ENABLED, settingsToApply='current')
# await latiss.rem.atarchiver.cmd_start.set_start(timeout=30)

In [5]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021060800001])

In [6]:
# Take 50 biases seq # 002-051
# Added wait to stop killing the recent images
for i in range(50):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, ski

In [7]:
# Take 10 10 second darks 52-61
await latiss.take_darks(10.0, 10)

Generating group_id
imagetype: DARK, skip TCS synchronization.
DARK 0001 - 0010
DARK 0002 - 0010
DARK 0003 - 0010
DARK 0004 - 0010
DARK 0005 - 0010
DARK 0006 - 0010
DARK 0007 - 0010
DARK 0008 - 0010
DARK 0009 - 0010
DARK 0010 - 0010


array([2021060800052, 2021060800053, 2021060800054, 2021060800055,
       2021060800056, 2021060800057, 2021060800058, 2021060800059,
       2021060800060, 2021060800061])

In [44]:
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [ ]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.ENABLED, settingsToApply='current')

In [53]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

logMessage DDS read queue is filling: 15 of 100 elements


In [36]:
await atcs.prepare_for_flatfield()

Enable dome trajectory following.
Cover state <MirrorCoverState.OPENED: 7>
M1 cover already opened.
Sending command
Stop tracking.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -042.000 deg; delta Az= -000.300 deg; delta N1 = +000.000 deg; delta N2 = -024.000 deg 
[Telescope] delta Alt = -040.735 deg; delta Az= +000.001 deg; delta N1 = -000.000 deg; delta N2 = -023.676 deg 
[Telescope] delta Alt = -034.808 deg; delta Az= +000.001 deg; delta N1 = -000.000 deg; delta N2 = -021.490 deg 
[Telescope] delta Alt = -028.805 deg; delta Az= +000.000 deg; delta N1 = -000.000 deg; delta N2 = -017.033 deg 
[Telescope] delta Alt = -022.829 deg; delta Az= -000.000 deg; delta N1 = -000.000 deg; delta N2 = -011.214 deg 
[Telescope] delta Alt = -018.935 deg; delta Az= -000.000 deg; delta N1 = -000.000 deg; delta N2 = -007.358 deg 
[Tel

RuntimeError: ATDome is deactivated. Activate it by setting `check.atdome=True` before slewing.In some cases users deactivate a component on purpose.Make sure it is clear to operate the dome before doing so.

In [41]:
# Take a test flat
await latiss.take_flats(2.0, 1, filter='RG610', grating='empty_1')

Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001


array([2021060800142])

In [34]:
# Pointing to a given Az/El  This will not track
az = 206.0
el = 81.0
rot = -86.0
await atcs.point_azel(az, el, rot_tel=rot)

Sending command
Stop tracking.
Tracking state: <AtMountState.TRACKINGENABLED: 9>
Tracking state: <AtMountState.STOPPING: 10>
In Position: True.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = +006.003 deg; delta Az= +000.000 deg; delta N1 = +000.000 deg; delta N2 = +000.000 deg 
[Telescope] delta Alt = +003.773 deg; delta Az= +000.000 deg; delta N1 = +000.000 deg; delta N2 = -000.000 deg 
[Telescope] delta Alt = +000.432 deg; delta Az= +000.000 deg; delta N1 = +000.000 deg; delta N2 = -000.000 deg 
Got True
Waiting for telescope to settle.
[Telescope] delta Alt = -000.000 deg; delta Az= +000.000 deg; delta N1 = -000.000 deg; delta N2 = -000.000 deg 
Telescope in position.


In [35]:
await atcs.stop_tracking()

Stop tracking.
Tracking state: <AtMountState.TRACKINGENABLED: 9>
Tracking state: <AtMountState.STOPPING: 10>
In Position: True.


In [30]:
# port=1 - Nasmyth 1
# port=2 - Nasmyth 2 
await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=2, timeout=30)

In [39]:
# Take 10 2 second flats 72-81
await latiss.take_flats(2.0, 10, filter='RG610', grating='empty_1')

Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0010
FLAT 0002 - 0010
logMessage DDS read queue is filling: 31 of 100 elements
FLAT 0003 - 0010
FLAT 0004 - 0010
FLAT 0005 - 0010
FLAT 0006 - 0010
FLAT 0007 - 0010
FLAT 0008 - 0010
FLAT 0009 - 0010
FLAT 0010 - 0010


array([2021060800072, 2021060800073, 2021060800074, 2021060800075,
       2021060800076, 2021060800077, 2021060800078, 2021060800079,
       2021060800080, 2021060800081])

In [40]:
# Take flats for PTC 82-141
# Added wait to stop killing the recent images
for i in range(30):
    exp = 0.2 * float(i+1)
    await latiss.take_flats(exp, 1, filter='RG610', grating='empty_1')
    if exp < 2.0:
        await asyncio.sleep(2.0)
    await latiss.take_flats(exp, 1, filter='RG610', grating='empty_1')


Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001
Generating group_id
imagetype: FLAT, ski

In [45]:
# This moves everything to park position and opens the dome.  It takes 5+ minutes.
await atcs.prepare_for_onsky()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for ataos.
Complete settings for atpneumatics.
Complete settings for athexapod.
Complete settings for atdome.
Complete settings for atdometrajectory.
Settings versions: {'atmcs': '', 'atptg': '', 'ataos': 'current', 'atpneumatics': '', 'athexapod': 'current', 'atdome': 'current', 'atdometrajectory': ''}
[atmcs]::[<State.ENABLED: 2>]
[atptg]::[<State.ENABLED: 2>]
[ataos]::[<State.ENABLED: 2>]
[atpneumatics]::[<State.ENABLED: 2>]
[athexapod]::[<State.ENABLED: 2>]
[atdome]::[<State.ENABLED: 2>]
[atdometrajectory]::[<State.ENABLED: 2>]
All components in <State.ENABLED: 2>.
Enable dome trajectory following.
Slew telescope to park position.
Sending command
Stop tracking.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <Sta

In [46]:
await atcs.rem.atpneumatics.cmd_closeM1Cover.start()

In [47]:
await atcs.home_dome()

Dome azimuth still homing.
Dome azimuth still homing.
Dome azimuth still homing.
Dome azimuth still homing.
Dome azimuth still homing.
Dome azimuth still homing.
Dome azimuth homed successfully.


In [49]:
atcs.check.atdome = True
atcs.check.atdometrajectory = True

In [50]:
await atcs.slew_dome_to(az=90.0)

Enable dome trajectory following.
process as completed...
atdometrajectory not in <State.DISABLED: 1>: <State.ENABLED: 2>


RuntimeError: atdometrajectory state is <State.ENABLED: 2>, expected <State.DISABLED: 1>

In [51]:
await atcs.open_dome_shutter()

Opening dome shutter...
process as completed...
atdome: <State.ENABLED: 2>
Waiting for ATDome mainDoorState: <ShutterDoorState.OPENED: 2>. Current state: <ShutterDoorState.CLOSED: 1>.
mainDoorState: <ShutterDoorState.OPENING: 4>
mainDoorState: <ShutterDoorState.OPENED: 2>
Finishing ATDome shutter command task.
ATDome shutter command task not done. Cancelling.
ATDome shutter command task cancelled.
logMessage DDS read queue is filling: 10 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements


In [ ]:
await atcs.rem.atpneumatics.cmd_openM1CellVents.start()

In [ ]:
# This Opens/Closes the dropout shutter
# await atcs.rem.atdome.cmd_moveShutterDropoutDoor.set_start(open=False)

In [ ]:
# This tunrs on ATAOS corrections and turns on the air pressure under the M1 mirror.
await atcs.rem.ataos.cmd_enableCorrection.set_start(
    m1=True, hexapod=True, atspectrograph=True, timeout=atcs.long_timeout
)

In [ ]:
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})

In [ ]:
await atcs.rem.atdometrajectory.cmd_setFollowingMode.set_start(enable=True)

In [ ]:
# Pointing to a given Az/El  This will not track
await atcs.point_azel(az, el, rot_tel=rot)

In [ ]:
# Pointing to a given RA/Dec  This will track
await atcs.slew_icrs(ra=ra, dec=dec, rot=rot, rot_type=RotType.PhysicalSky)

In [60]:
# Slew to a given object and start tracking.
await atcs.slew_object('HD 187101', rot_type=RotType.Parallactic)

Resetting dropped connection: simbad.u-strasbg.fr
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HD 187101: 19 51 23.6559 -58 30 35.538
Setting rotator position with respect to parallactic angle to 0.0 deg.
Parallactic angle: -64.28825482713914 | Sky Angle: 25.711745172860844
Sending command
Stop tracking.
target python read queue is filling: 26 of 100 elements
Tracking state: <AtMountState.TRACKINGENABLED: 9>
Tracking state: <AtMountState.STOPPING: 10>
In Position: True.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +001.590 deg; delta Az = +011.869 deg; delta N1 = -000.000 deg; delta N2 = +002.018 deg [Dome] delta Az = +008.832 deg
[Telescope] delta Alt = +000.916 deg; delta Az = +010.487 deg; delta N1 = -000.000 de

In [57]:
# It is recommended to do this and wait several seconds for the spectrograph to settle
# before taking an image
await latiss.setup_atspec(filter='RG610', grating='empty_1')

In [66]:
# Take 1 2 second image
await latiss.take_object(30.0, 1, filter='RG610', grating='empty_1')

Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001


array([2021060800390])

logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 10 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 12 of 100 elements
logMessage DDS read queue is filling: 12 of 100 elements
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 17 of 100 elements
logMessage DDS read queue is filling: 14 of 100 elements


In [ ]:
await atcs.rem.ataos.cmd_disableCorrection.set_start(atspectrograph=True)

In [54]:
# This loop will take a set of images on both sides of current best focus
# starts at starting_z_offset and runs to -starting_z_offset
# then puts the z_offset back where it was.

starting_z_offset = -0.20
z_offset_increment = 0.05
nsteps = int((-2 * starting_z_offset) / z_offset_increment) + 1
total_z_offset = 0.0
await atcs.rem.ataos.cmd_offset.set_start(z=starting_z_offset)
total_z_offset += starting_z_offset
print(f"Total z offset = {total_z_offset}")
await asyncio.sleep(2)
for i in range(nsteps):
    await latiss.take_object(5.0, 1, filter='RG610', grating='holo4_003')
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_increment)
    total_z_offset += z_offset_increment
    print(f"Total z offset = {total_z_offset}")
    
# Put offset back where it was
await atcs.rem.ataos.cmd_offset.set_start(z=-total_z_offset)
total_z_offset -= total_z_offset
print(f"Total z offset = {total_z_offset}")
    

Total z offset = -0.2
Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001
Total z offset = -0.15000000000000002
Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001
Total z offset = -0.10000000000000002
Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001
Total z offset = -0.05000000000000002
Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001
Total z offset = -1.3877787807814457e-17
Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001
Total z offset = 0.04999999999999999
Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001
Total z offset = 0.09999999999999999
Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001
Total z offset = 0.15
Generating group_id
imagetype: OBJECT, TCS synchronization not config

In [ ]:
# This will put in a hexapod offset
#await atcs.rem.ataos.cmd_offset.set_start(z=-0.095)

In [ ]:
# To reset all hexapod offsets
tmp = await atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')

In [64]:
# Move the star within the field
# Offsets are in arcseconds.
await atcs.offset_xy(y=-100, x=0, relative=True)

Calculating x/y offset: 0/-100 
Applying Az/El offset: 99.99559130587656/0.9389991416958944 
Telescope not in position.
All axes in position.
Waiting for telescope to settle.
Done


In [ ]:
await latiss.take_object(5.0, 1, filter='RG610', grating='ronchi170lpmm')

In [ ]:
atcs.offset_azel?

In [ ]:
# Run a figure-8 of offsets
offsets = [[0,0], [0,100], [100,0], [0,-100], [-100,0], [0,-100], [-100,0], [0,100], [100,0]]
await latiss.setup_atspec(filter='RG610', grating='empty_1')
await asyncio.sleep(2)
for [xx, yy] in offsets:
    await atcs.offset_azel(az=xx, el=yy, relative=True)
    await asyncio.sleep(2)
    await latiss.take_object(2.0, 1, filter='RG610', grating='empty_1')
    await asyncio.sleep(2)
    

In [ ]:
# To enable or disable built-in offsets for the filters and gratings
#await atcs.rem.ataos.cmd_disableCorrection.set_start(atspectrograph=True)
#await atcs.rem.ataos.cmd_enableCorrection.set_start(atspectrograph=True)

In [ ]:
# To stop tracking
# await atcs.stop_tracking()

In [ ]:
# If you are done, put things in standby, or shut things down completely
#await atcs.standby()
await atcs.shutdown()

In [67]:
# Putting everything back in standby.
await latiss.standby()

[atcamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atspectrograph]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.
